In [1]:
# Import necessary libraries

import sqlite3

In [2]:
# Connect to databse
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Part 3: SQL

## Assume we are using these two tables:

## What are the problem(s) with this SQL query?

- __WHERE order.order_date >= '20090101'__
    - In the orders table the format of the column order_date is, 'year-month-day'. The format of the date in the SQL query is 'yearmonthday'. These two formats do not match. This results in every row being returned that does not have a null value.
- __SELECT ... SUM(COALESCE(orders.order_amt, 0)) AS total_2019__
    - Using COALESCE does not make any sense in this scenario. The order of operations says the FROM and WHERE statements are executed before the SELECT statement. Therefore, there is no need to account for any null values in the orders_amt column because any rows that have null values through the join (Julie Peters) will be dropped during the WHERE statement because the date will be null as well. It would be simpler to use the statement __SUM(orders.orders_amt).

In [160]:
c.execute('CREATE TABLE customers (customer_nbr int, customer_name varchar(255))')
conn.commit()

In [161]:
c.execute("INSERT INTO customers (customer_nbr, customer_name) VALUES (1, 'Jim Brown')")
c.execute("INSERT INTO customers (customer_nbr, customer_name) VALUES (2, 'Jeff Gordon')")
c.execute("INSERT INTO customers (customer_nbr, customer_name) VALUES (3, 'Peter Green')")
c.execute("INSERT INTO customers (customer_nbr, customer_name) VALUES (4, 'Julie Peters')")

In [164]:
c.execute("""SELECT *
            FROM customers""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_nbr,customer_name
0,1,Jim Brown
1,2,Jeff Gordon
2,3,Peter Green
3,4,Julie Peters


In [170]:
c.execute('CREATE TABLE orders (order_nbr int, order_date datetime, customer_nbr int, order_amt int)')
conn.commit()

In [171]:
c.execute("INSERT INTO orders (order_nbr, order_date, customer_nbr, order_amt) VALUES (1, '2008-10-01', 1, 15.50)")
c.execute("INSERT INTO orders (order_nbr, order_date, customer_nbr, order_amt) VALUES (2, '2008-12-15', 2, 25.50)")
c.execute("INSERT INTO orders (order_nbr, order_date, customer_nbr, order_amt) VALUES (3, '2009-01-02', 1, 18.00)")
c.execute("INSERT INTO orders (order_nbr, order_date, customer_nbr, order_amt) VALUES (4, '2009-02-20', 3, 10.25)")
c.execute("INSERT INTO orders (order_nbr, order_date, customer_nbr, order_amt) VALUES (5, '2009-03-05', 1, 30.00)")

In [181]:
c.execute("""SELECT *
            FROM orders""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,order_nbr,order_date,customer_nbr,order_amt
0,1,2008-10-01,1,15.50
1,2,2008-12-15,2,25.50
2,3,2009-01-02,1,18.00
3,4,2009-02-20,3,10.25
4,5,2009-03-05,1,30.00


In [182]:
c.execute("""SELECT customers.customer_name,
                    SUM(COALESCE(orders.order_amt, 0)) AS total_2009
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
            WHERE orders.order_date >= '20090101'
            GROUP BY customers.customer_name""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_name,total_2009
0,Jeff Gordon,25.50
1,Jim Brown,63.50
2,Peter Green,10.25


In [183]:
c.execute("""SELECT *
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_nbr,customer_name,order_nbr,order_date,customer_nbr,order_amt
0,1,Jim Brown,1.0,2008-10-01,1.0,15.50
1,1,Jim Brown,3.0,2009-01-02,1.0,18.00
2,1,Jim Brown,5.0,2009-03-05,1.0,30.00
3,2,Jeff Gordon,2.0,2008-12-15,2.0,25.50
4,3,Peter Green,4.0,2009-02-20,3.0,10.25
5,4,Julie Peters,NaN,None,NaN,NaN


In [180]:
c.execute("""SELECT *
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
            WHERE orders.order_date >= '20090101'""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_nbr,customer_name,order_nbr,order_date,customer_nbr,order_amt
0,1,Jim Brown,1,2008-10-01,1,15.50
1,2,Jeff Gordon,2,2008-12-15,2,25.50
2,1,Jim Brown,3,2009-01-02,1,18.00
3,3,Peter Green,4,2009-02-20,3,10.25
4,1,Jim Brown,5,2009-03-05,1,30.00


In [185]:
c.execute("""SELECT *, COALESCE(orders.order_amt, 0) AS total_2009
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
            WHERE orders.order_date >= '20090101'""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_nbr,customer_name,order_nbr,order_date,customer_nbr,order_amt,total_2009
0,1,Jim Brown,1,2008-10-01,1,15.50,15.50
1,2,Jeff Gordon,2,2008-12-15,2,25.50,25.50
2,1,Jim Brown,3,2009-01-02,1,18.00,18.00
3,3,Peter Green,4,2009-02-20,3,10.25,10.25
4,1,Jim Brown,5,2009-03-05,1,30.00,30.00


In [187]:
c.execute("""SELECT customers.customer_name, SUM(COALESCE(orders.order_amt, 0)) AS total_2009
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
            WHERE orders.order_date >= '20090101'
            GROUP BY customers.customer_name""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_name,total_2009
0,Jeff Gordon,25.50
1,Jim Brown,63.50
2,Peter Green,10.25


In [188]:
c.execute("""SELECT customers.customer_name, SUM(orders.order_amt) AS total_2009
            FROM customers
            LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
            WHERE orders.order_date >= '20090101'
            GROUP BY customers.customer_name""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data

,customer_name,total_2009
0,Jeff Gordon,25.50
1,Jim Brown,63.50
2,Peter Green,10.25
